In [1]:
import naslib.optimizers

ModuleNotFoundError: No module named 'naslib.search_spaces.simple_cell'

In [2]:
import logging
import sys

from naslib.defaults.trainer import Trainer

from naslib.search_spaces import (
    DartsSearchSpace,
    SimpleCellSearchSpace,
    NasBench101SearchSpace,
    HierarchicalSearchSpace,
)
from naslib.search_spaces.nasbench101 import graph

from naslib.utils import get_dataset_api, setup_logger, utils

import numpy as np

ModuleNotFoundError: No module named 'naslib.defaults'

### Defining the Dataset

In [2]:
config = utils.get_config_from_args(config_type="nas")

In [3]:
dataset_api = get_dataset_api(config.search_space, config.dataset)

Loading dataset from file... This may take a few minutes...
Loaded dataset in 6 seconds


### Defining the Cell Configuration

In [4]:
INPUT = "input"
OUTPUT = "output"
CONV3X3 = "conv3x3-bn-relu"
CONV1X1 = "conv1x1-bn-relu"
MAXPOOL3X3 = "maxpool3x3"
OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]

NUM_VERTICES = 7
OP_SPOTS = NUM_VERTICES - 2
MAX_EDGES = 9

In [5]:
def sample_random_architecture(dataset_api, arch_limit=10):
        """
        This will sample a random architecture and update the edges in the
        naslib object accordingly.
        From the NASBench repository:
        one-hot adjacency matrix
        draw [0,1] for each slot in the adjacency matrix
        """
        architectures = []
        while len(architectures) < arch_limit:
            matrix = np.random.choice([0, 1], size=(NUM_VERTICES, NUM_VERTICES))
            matrix = np.triu(matrix, 1)
            ops = np.random.choice(OPS, size=NUM_VERTICES).tolist()
            ops[0] = INPUT
            ops[-1] = OUTPUT
            spec = dataset_api["api"].ModelSpec(matrix=matrix, ops=ops)
            if dataset_api["nb101_data"].is_valid(spec):
                architectures.append({"matrix": matrix, "ops": ops})
                #break
        
        return architectures
            
        #self.set_spec({"matrix": matrix, "ops": ops})

### Sampling the architectures

In [6]:
sampled_architectures = sample_random_architecture(dataset_api)

In [9]:
sampled_architectures

[{'matrix': array([[0, 1, 0, 1, 1, 0, 1],
         [0, 0, 1, 0, 0, 1, 1],
         [0, 0, 0, 0, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',
   'conv3x3-bn-relu',
   'conv3x3-bn-relu',
   'maxpool3x3',
   'conv1x1-bn-relu',
   'maxpool3x3',
   'output']},
 {'matrix': array([[0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 1, 1],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 0],
         [0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',
   'conv1x1-bn-relu',
   'maxpool3x3',
   'maxpool3x3',
   'maxpool3x3',
   'conv1x1-bn-relu',
   'output']},
 {'matrix': array([[0, 0, 1, 0, 0, 0, 1],
         [0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',
   'maxpoo

In [10]:
sampled_architectures[0]

{'matrix': array([[0, 1, 0, 1, 1, 0, 1],
        [0, 0, 1, 0, 0, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]),
 'ops': ['input',
  'conv3x3-bn-relu',
  'conv3x3-bn-relu',
  'maxpool3x3',
  'conv1x1-bn-relu',
  'maxpool3x3',
  'output']}

### Converting the architecture into Pytorch Neural Network

In [7]:
from naslib.predictors.utils.models import nasbench1 as nas101_arch
from naslib.predictors.utils.models import nasbench1_spec
import torchvision.transforms as transforms
import torchvision.datasets as dset
from pathlib import Path
import torch.nn.functional as F
from tqdm import tqdm
from time import sleep
import torch

In [9]:
spec = nasbench1_spec._ToModelSpec(
    sampled_architectures[0]["matrix"], sampled_architectures[0]["ops"]
)

num_classes_dic = {"cifar10": 10, "cifar100": 100, "ImageNet16-120": 120}

network = nas101_arch.Network(
    spec,
    stem_out=128,
    num_stacks=3,
    num_mods=3,
    num_classes=num_classes_dic["cifar10"]
)

### Exporting the Architecture

In [10]:
torch.save(network, './network0')

### Importing the Architecture

In [11]:
network_1 = torch.load('./network0')

### Checking if the exported and the imported architectures are the same

In [32]:
def check_model_equality(model_1, model_2):
    for p1, p2 in zip(model_1.parameters(), model_2.parameters()):
        if p1.data.ne(p2.data).sum() > 0:
            return False
    
    return True


In [33]:
check_model_equality(network, network_1)

True

### Preparing the network for `CIFAR10` classification

In [8]:
def get_project_root() -> Path:
    """
    Returns the root path of the project.
    """
    return Path('./').parent.parent

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
on_device = network.to(device)

In [ ]:
type(on_device)

naslib.predictors.utils.models.nasbench1.Network

### Preparing data transformations for `CIFAR10`

In [ ]:
class Cutout(object):
    def __init__(self, length, prob=1.0):
        self.length = length
        self.prob = prob

    def __call__(self, img):
        if np.random.binomial(1, self.prob):
            h, w = img.size(1), img.size(2)
            mask = np.ones((h, w), np.float32)
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1:y2, x1:x2] = 0.0
            mask = torch.from_numpy(mask)
            mask = mask.expand_as(img)
            img *= mask
        return img


In [ ]:

CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

train_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ]
)

train_transform.transforms.append(Cutout(16, 1))

valid_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ]
)


In [ ]:
data = "{}/data".format(get_project_root())
seed = 42

In [ ]:
train_data = dset.CIFAR10(
    root=data, train=True, download=True, transform=train_transform
)
test_data = dset.CIFAR10(
    root=data, train=False, download=True, transform=valid_transform
)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(0.8 * num_train))
batch_size = 64

train_queue = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
    pin_memory=True,
    num_workers=0,
    worker_init_fn=np.random.seed(seed+1),
)

valid_queue = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[split:num_train]),
    pin_memory=True,
    num_workers=0,
    worker_init_fn=np.random.seed(seed),
)

test_queue = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=0,
    worker_init_fn=np.random.seed(seed),
)

### Defining optimizer, lr, scheduler, loss function

In [ ]:
loss_fn = F.cross_entropy
lr = 0.4
optimizer = torch.optim.SGD(network.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


### Training the network

In [ ]:
network.train()
for epoch in range(1, 5):
    with tqdm(train_queue, unit="batch") as tepoch:
        for data, target in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = network(data)
            predictions = output.argmax(dim=1, keepdim=True).squeeze()
            loss = F.nll_loss(output, target)
            correct = (predictions == target).sum().item()
            accuracy = correct / batch_size
            
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
            sleep(0.1)

Epoch 2:  29%|██▉       | 184/625 [05:14<12:18,  1.68s/batch, accuracy=9.38, loss=nan]